In [1]:
import pandas as pd

In [2]:
game_df = pd.read_csv('UserGameplayData.csv')

In [3]:
game_df.head()

,User ID,Games Played,Datetime
0,851,1,01-10-2022 00:00
1,717,1,01-10-2022 00:00
2,456,1,01-10-2022 00:00
3,424,1,01-10-2022 00:00
4,845,1,01-10-2022 00:00


In [4]:
game_df.rename(columns = {'User ID' : 'User Id'}, inplace = True)

In [5]:
game_df.isnull().sum()

User Id         0
Games Played    0
Datetime        0
dtype: int64

In [6]:
game_df.shape

(355266, 3)

In [7]:
deposit_df = pd.read_csv('DepositData.csv')

In [8]:
deposit_df.head()

,User Id,Datetime,Amount
0,357,01-10-2022 00:03,2000
1,776,01-10-2022 00:03,2500
2,492,01-10-2022 00:06,5000
3,803,01-10-2022 00:07,5000
4,875,01-10-2022 00:09,1500


In [9]:
deposit_df.shape

(17438, 3)

In [10]:
deposit_df.isnull().sum()

User Id     0
Datetime    0
Amount      0
dtype: int64

In [11]:
deposit_df.rename(columns = {'Amount' : 'Deposit'}, inplace = True)

In [12]:
withdrawal_df = pd.read_csv('WithdrawalData.csv')

In [13]:
withdrawal_df.head()

,User Id,Datetime,Amount
0,190,01-10-2022 00:03,5872
1,159,01-10-2022 00:16,9540
2,164,01-10-2022 00:24,815
3,946,01-10-2022 00:29,23000
4,763,01-10-2022 00:40,9473


In [14]:
withdrawal_df.shape

(3566, 3)

In [15]:
withdrawal_df.isnull().sum()

User Id     0
Datetime    0
Amount      0
dtype: int64

In [16]:
withdrawal_df.rename(columns = {'Amount' : 'Withdrawal'}, inplace = True)

In [17]:
withdrawal_df['Datetime'] = pd.to_datetime(withdrawal_df['Datetime'],format='%d-%m-%Y %H:%M')

In [18]:
deposit_df['Datetime'] = pd.to_datetime(deposit_df['Datetime'],format='%d-%m-%Y %H:%M')

In [19]:
game_df['Datetime'] = pd.to_datetime(game_df['Datetime'],format='%d-%m-%Y %H:%M')

In [20]:
# separate date from date time and creating slot column

In [21]:
def slot(hour):
    return "S1" if hour<12 else "S2"

for df in [game_df,deposit_df, withdrawal_df]:
    df['Date'] = df['Datetime'].dt.date
    df['Slot'] = df['Datetime'].dt.hour.apply(slot)

In [22]:
df.head()

,User Id,Datetime,Withdrawal,Date,Slot
0,190,2022-10-01 00:03:00,5872,2022-10-01,S1
1,159,2022-10-01 00:16:00,9540,2022-10-01,S1
2,164,2022-10-01 00:24:00,815,2022-10-01,S1
3,946,2022-10-01 00:29:00,23000,2022-10-01,S1
4,763,2022-10-01 00:40:00,9473,2022-10-01,S1


In [23]:
game = game_df.groupby(['User Id','Date','Slot'])['Games Played'].sum().reset_index()

In [24]:
deposit = deposit_df.groupby(['User Id','Date','Slot']).agg(Deposit_Amount=('Deposit','sum'),Deposit_Count=('Deposit','count')).reset_index()

In [25]:
withdrawal = withdrawal_df.groupby(['User Id','Date','Slot']).agg(withdrawal_Amount=('Withdrawal','sum'),withdrawal_Count=('Withdrawal','count')).reset_index()

In [26]:
merged_df = pd.merge(game,deposit,on=['User Id','Date','Slot'],how='outer')

In [27]:
merged_df = pd.merge(merged_df,withdrawal,on=['User Id','Date','Slot'],how='outer')

In [28]:
merged_df.fillna(0,inplace=True)

In [29]:
merged_df.tail()

,User Id,Date,Slot,Games Played,Deposit_Amount,Deposit_Count,withdrawal_Amount,withdrawal_Count
39046,980,2022-10-19,S1,0.0,0.0,0.0,800.0,1.0
39047,980,2022-10-19,S2,0.0,0.0,0.0,800.0,1.0
39048,980,2022-10-23,S1,0.0,0.0,0.0,1200.0,1.0
39049,980,2022-10-28,S1,0.0,0.0,0.0,200.0,1.0
39050,980,2022-10-31,S1,0.0,0.0,0.0,1100.0,1.0


In [30]:
merged_df.isnull().sum()

User Id              0
Date                 0
Slot                 0
Games Played         0
Deposit_Amount       0
Deposit_Count        0
withdrawal_Amount    0
withdrawal_Count     0
dtype: int64

In [31]:
merged_df['withdrawal_Count'].value_counts()

withdrawal_Count
0.0    35948
1.0     2722
2.0      314
3.0       57
4.0        7
5.0        2
7.0        1
Name: count, dtype: int64

### PART A - Calculating Loyalty Point

In [32]:
# Creating a function to find the Loyality point

In [33]:
def loyalty_point_by_slot (merged_df,date_str,slot):
    weights = {
        'deposit_amt':0.01,
        'withdrawal_amt' : 0.005,
        'deposit_count_minus_withdrawal_count' : 0.001,
        'games_played' : 0.2
    }
    date = pd.to_datetime(date_str).date()
    
    # creating a data frame to filter out the data as per the slot and date
    slot_date_df = merged_df[
        (merged_df['Date'].dt.date == date) &
        (merged_df['Slot'] == slot)
    ].copy()
    slot_date_df.fillna(0,inplace=True)
    
    # calculating the Loyalty point
    slot_date_df['Loyalty Point'] = (
        weights['deposit_amt']*slot_date_df['Deposit_Amount']+
        weights['withdrawal_amt']*slot_date_df['withdrawal_Amount']+
        weights['deposit_count_minus_withdrawal_count']*(slot_date_df['Deposit_Count']-slot_date_df['withdrawal_Count']).clip(lower=0)+
        weights['games_played']*slot_date_df['Games Played']
    )

    # return loyalty points
    return slot_date_df.sort_values(by='Loyalty Point',ascending=False)

In [34]:
merged_df['Date'] = pd.to_datetime(merged_df['Date'],errors='coerce')

In [35]:
slots = [
    ("2022-10-02","S1"),
    ("2022-10-16","S2"),
    ("2022-10-18","S1"),
    ("2022-10-26","S2")
]

In [36]:
for date_str, slot in slots:
    print(f'\n date {date_str}...... Slot {slot} ')
    result = loyalty_point_by_slot(merged_df,date_str,slot)
    print(result[['User Id','Loyalty Point']] .head())


 date 2022-10-02...... Slot S1 
       User Id  Loyalty Point
38816      634       1478.355
36894      672       1300.000
18590      566       1250.406
38093      949        677.500
14334      446        550.202

 date 2022-10-16...... Slot S2 
       User Id  Loyalty Point
38826      634       1491.555
35019      212        999.991
34395       99        980.002
34088       28        900.004
18618      566        880.203

 date 2022-10-18...... Slot S1 
       User Id  Loyalty Point
38827      634       2723.100
6516       208       1701.401
22617      673        900.801
34759      162        770.000
38605      245        750.000

 date 2022-10-26...... Slot S2 
       User Id  Loyalty Point
37144      714       2000.001
11627      369       1501.915
21083      634       1237.010
17583      538       1200.403
34017        2        900.002


In [37]:
#calculating the loyalty point of players in the month of october and give ranking

In [38]:
october_data = merged_df[
    (merged_df['Date'].dt.year ==2022) &
    (merged_df['Date'].dt.month == 10)
].copy()

In [39]:
results = []

for _, row in october_data[['Date','Slot']].drop_duplicates().iterrows():
    date_str = row['Date'].strftime('%Y-%m-%d')
    slot = row['Slot']
    df_slot =loyalty_point_by_slot(merged_df,date_str,slot)
    results.append(df_slot[['User Id','Loyalty Point','Games Played']])

In [40]:
combined = pd.concat(results)

In [41]:
final_result = combined.groupby('User Id').agg({
    "Loyalty Point" : "sum",
    "Games Played" : "sum"
}).reset_index()

In [42]:
final_result =final_result.sort_values(by=['Loyalty Point','Games Played'],ascending=[False,False])
final_result['Rank'] = range(1,len(final_result)+1)

In [43]:
print(final_result[['Rank','User Id','Loyalty Point','Games Played']].head())

     Rank  User Id  Loyalty Point  Games Played
634     1      634      83843.326          24.0
99      2       99      23665.744          10.0
672     3      672      22757.783          10.0
212     4      212      22199.286           1.0
740     5      740      19211.826           2.0


In [44]:
#Calculating Average Deposit amount

In [45]:
average_deposit = deposit_df['Deposit'].mean()

In [46]:
print("Average Deposit Amount : ",round(average_deposit,2))

Average Deposit Amount :  5492.19


In [47]:
# Average deposit amount per user in a month.

In [48]:
deposit_df['Datetime'] = pd.to_datetime(deposit_df['Datetime'])

In [49]:
deposit_df['YearMonth'] = deposit_df['Datetime'].dt.to_period('M')

In [50]:
monthly_deposit = deposit_df.groupby(['User Id','YearMonth'])['Deposit'].sum().reset_index()

In [51]:
average_monthly_deposit = monthly_deposit.groupby('User Id')['Deposit'].mean().reset_index()

In [52]:
average_monthly_deposit.columns = ['User Id','Avg Monthly Deposit']

In [53]:
average_monthly_deposit.head()

,User Id,Avg Monthly Deposit
0,1,5000.0
1,2,567000.0
2,3,40000.0
3,4,1750.0
4,5,74100.0


In [54]:
# Average number of games playedby user.

In [55]:
game_df['Datetime'] = pd.to_datetime(game_df['Datetime'])

In [56]:
game_per_user = game_df.groupby('User Id')['Games Played'].sum().reset_index()

In [57]:
average_game_per_user = game_per_user['Games Played'].mean()

In [58]:
print('Average number of games played by user : ',round(average_game_per_user,2))

Average number of games played by user :  355.27


### Part B

In [59]:
# calculating the bonus for first 50 players based on there loyalty point

In [60]:
top_50 = final_result.sort_values(by=['Loyalty Point','Games Played'],ascending=[False,False]).head(50).copy()

In [61]:
total_loyalty_point = top_50['Loyalty Point'].sum()

In [62]:
top_50['Bonus'] = (top_50['Loyalty Point']/total_loyalty_point)*50000

In [63]:
top_50['Bonus'] = top_50['Bonus'].round(2)

In [64]:
top_50[['Rank','User Id','Loyalty Point','Games Played','Bonus']].head()

,Rank,User Id,Loyalty Point,Games Played,Bonus
634,1,634,83843.326,24.0,6638.86
99,2,99,23665.744,10.0,1873.89
672,3,672,22757.783,10.0,1802.00
212,4,212,22199.286,1.0,1757.78
740,5,740,19211.826,2.0,1521.23


### Part C